In [2]:
import pandas as pd
import numpy as np
from vosk import Model, KaldiRecognizer, SetLogLevel
import sys
import os
import wave
import subprocess
import json
from os import listdir
from os.path import isfile, join
import datetime
import time

In [6]:
start = time.time()

files=pd.DataFrame(columns=['folder','path_audio'])

dir_audios="../Data/Audio"
dir_audios= dir_audios+"/" if dir_audios[-1]!="/" else dir_audios
folders_audio = [dir_audios+f for f in listdir(dir_audios)]

for folders in folders_audio:
    file_audio = [folders+"/"+f for f in listdir(folders)]
    file_audio = pd.DataFrame(file_audio,columns=['path_audio'])
    file_audio['folder'] = folders.split("/")[-1]

    files = files.append(file_audio)
files.head()

,folder,path_audio
0,C5N,../Data/Audio/C5N/C5N2020-08-14-17-55-49.wav
0,TN,../Data/Audio/TN/TN2020-08-14-18-59-25.wav


In [7]:
files['file']=files['path_audio'].apply(lambda x: x.split("/")[-1])
aux = pd.Series(files['file'].apply(lambda x: x.replace(".wav","").split("-")[-6:]))
aux = aux.apply(lambda x: [int(a) if len(a)<=4 else int(a[-4:]) for a in x])
files['date'] = aux.apply(lambda x: datetime.datetime(x[0], x[1], x[2], hour=x[3], minute=x[4], second=x[5]))

files.reset_index(drop=True, inplace=True)
files

,folder,path_audio,file,date
0,C5N,../Data/Audio/C5N/C5N2020-08-14-17-55-49.wav,C5N2020-08-14-17-55-49.wav,2020-08-14 17:55:49
1,TN,../Data/Audio/TN/TN2020-08-14-18-59-25.wav,TN2020-08-14-18-59-25.wav,2020-08-14 18:59:25


In [8]:
folder_model="../Data/Modelos/audio_model_spanish_vosk"
model = Model(folder_model)

registros = pd.DataFrame(columns=['path_audio','result','text'])
for file in files['path_audio']:
    wf = wave.open(file, "rb")
    rec = KaldiRecognizer(model, wf.getframerate())    

    while True:
        data = wf.readframes(1000000)
        if len(data) == 0:
            break
        if rec.AcceptWaveform(data):
            try:
                new_input = json.loads(rec.Result())
                new_input = pd.DataFrame(new_input)
                new_input['path_audio']=file
                registros = registros.append(new_input)
            except ValueError:
                 continue
        else:
            rec.PartialResult()


In [9]:
registros.reset_index(drop=True,inplace=True)
registros_2 = registros.join(pd.DataFrame(x for x in registros['result']))
registros_2.drop(columns='result',inplace=True)
registros_2.head()

,path_audio,text,conf,end,start,word
0,../Data/Audio/C5N/C5N2020-08-14-17-55-49.wav,hola hola muy buenas tardes bienvenidos al dia...,1.0,25.26,25.08,hola
1,../Data/Audio/C5N/C5N2020-08-14-17-55-49.wav,hola hola muy buenas tardes bienvenidos al dia...,1.0,25.41,25.26,hola
2,../Data/Audio/C5N/C5N2020-08-14-17-55-49.wav,hola hola muy buenas tardes bienvenidos al dia...,1.0,25.71,25.41,muy
3,../Data/Audio/C5N/C5N2020-08-14-17-55-49.wav,hola hola muy buenas tardes bienvenidos al dia...,1.0,25.98,25.71,buenas
4,../Data/Audio/C5N/C5N2020-08-14-17-55-49.wav,hola hola muy buenas tardes bienvenidos al dia...,1.0,26.31,25.98,tardes


In [10]:
MinTime = pd.DataFrame(registros_2.groupby(['path_audio','text'])['start'].min())
MaxTime = pd.DataFrame(registros_2.groupby(['path_audio','text'])['end'].max())

registros_3=registros_2[['path_audio','text']]
registros_3 = registros_3.merge(MinTime,on=['path_audio','text'])
registros_3 = registros_3.merge(MaxTime,on=['path_audio','text'])

registros_3.drop_duplicates(inplace=True)
registros_3.reset_index(drop=True,inplace=True)


registros_3.head()

,path_audio,text,start,end
0,../Data/Audio/C5N/C5N2020-08-14-17-55-49.wav,hola hola muy buenas tardes bienvenidos al dia...,25.08,44.94
1,../Data/Audio/C5N/C5N2020-08-14-17-55-49.wav,tenemos que hablar de esta definición por part...,45.24,90.24
2,../Data/Audio/C5N/C5N2020-08-14-17-55-49.wav,algún punto empieza a reconocer también y si u...,90.30,135.66
3,../Data/Audio/C5N/C5N2020-08-14-17-55-49.wav,hay una circulación realmente muy alta como es...,135.66,181.02
4,../Data/Audio/C5N/C5N2020-08-14-17-55-49.wav,en esta jornada tan importante hablamos de esp...,181.20,226.29


In [11]:
data = files.merge(registros_3, on='path_audio')

data['start'] = data['start'] .apply(lambda x: datetime.timedelta(seconds=int(x)))
data['start'] = data['start'] + data['date']
data['end'] = data['end'] .apply(lambda x: datetime.timedelta(seconds=int(x)))
data['end'] = data['end'] + data['date']

print(data.shape)
data.head()

(317, 7)


,folder,path_audio,file,date,text,start,end
0,C5N,../Data/Audio/C5N/C5N2020-08-14-17-55-49.wav,C5N2020-08-14-17-55-49.wav,2020-08-14 17:55:49,hola hola muy buenas tardes bienvenidos al dia...,2020-08-14 17:56:14,2020-08-14 17:56:33
1,C5N,../Data/Audio/C5N/C5N2020-08-14-17-55-49.wav,C5N2020-08-14-17-55-49.wav,2020-08-14 17:55:49,tenemos que hablar de esta definición por part...,2020-08-14 17:56:34,2020-08-14 17:57:19
2,C5N,../Data/Audio/C5N/C5N2020-08-14-17-55-49.wav,C5N2020-08-14-17-55-49.wav,2020-08-14 17:55:49,algún punto empieza a reconocer también y si u...,2020-08-14 17:57:19,2020-08-14 17:58:04
3,C5N,../Data/Audio/C5N/C5N2020-08-14-17-55-49.wav,C5N2020-08-14-17-55-49.wav,2020-08-14 17:55:49,hay una circulación realmente muy alta como es...,2020-08-14 17:58:04,2020-08-14 17:58:50
4,C5N,../Data/Audio/C5N/C5N2020-08-14-17-55-49.wav,C5N2020-08-14-17-55-49.wav,2020-08-14 17:55:49,en esta jornada tan importante hablamos de esp...,2020-08-14 17:58:50,2020-08-14 17:59:35


In [12]:
data_text_complete  = pd.DataFrame(columns=['path_audio','text_complete'])
for file in data['path_audio'].unique():
    text = data.loc[data['path_audio'] == file,'text'].unique()
    text_complete = ','.join(text)
    new_input=pd.DataFrame([text_complete], columns=['text_complete'])
    new_input['path_audio'] = file
    data_text_complete = data_text_complete.append(new_input)

data_text_complete.reset_index(drop=True,inplace=True)
data_text_complete=data[['folder','path_audio','file','date']].drop_duplicates().merge(data_text_complete, on = 'path_audio')

In [13]:
pd.set_option('display.max_colwidth',50) ##el segundo parametro indica la cantidad de caracteres a mostrar, con valor -1 muestra todo
data_text_complete

,folder,path_audio,file,date,text_complete
0,C5N,../Data/Audio/C5N/C5N2020-08-14-17-55-49.wav,C5N2020-08-14-17-55-49.wav,2020-08-14 17:55:49,hola hola muy buenas tardes bienvenidos al dia...
1,TN,../Data/Audio/TN/TN2020-08-14-18-59-25.wav,TN2020-08-14-18-59-25.wav,2020-08-14 18:59:25,de los colores que se habían ido arrojó de las...


In [14]:
##Export datasets
data.to_csv("../Data procesada/data_audio.csv")
data_text_complete.to_csv("../Data procesada/data_audio_concatenada.csv")

In [15]:
end = time.time()
print("Tiempo de procesamiento:",round((end - start)/60),"minutos")

Tiempo de procesamiento: 641 minutos
